# Näidis Hotelli ja Lennupiletite Broneerimise Agent

See lahendus aitab teil broneerida lennupileteid ja hotelli. Stsenaarium on reis Londoni Heathrow (LHR) lennujaamast 20. veebruaril 2024 New Yorki (JFK), tagasilend 27. veebruaril 2025, majandusklassis ainult British Airwaysiga. Soovin peatuda Hiltoni hotellis New Yorgis, palun esitage lennu- ja hotellikulud.


# Azure AI Agent Service'i käivitamine ja konfiguratsiooniteabe hankimine **.env** failist

### **.env**

Loo .env fail

**.env** sisaldab Azure AI Agent Service'i ühenduse stringi, AOAI poolt kasutatavat mudelit ning vastavat Google API otsinguteenuse API-d, ENDPOINT-i jne.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Teie Azure AI Agent Service'i mudeli juurutamise nimi"

[**NOTE**] Teil on vaja mudelit, millel on 100,000 Rate Limit (Tokenid minutis) ja Rate Limit 600 (Päringud minutis).

  Mudeli ja Endpointi saate Azure AI Foundry - Model and Endpoint sektsioonist.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Teie Azure AI Agent Service'i projekti ühenduse string"

  Projekti ühenduse stringi leiate oma projekti ülevaatest AI Foundry portaali ekraanil.

- **SERPAPI_SEARCH_API_KEY** = "Teie SERPAPI otsinguteenuse API võti"
- **SERPAPI_SEARCH_ENDPOINT** = "Teie SERPAPI otsinguteenuse Endpoint"

Azure AI Agent Service'i mudeli juurutamise nime ja projekti ühenduse stringi hankimiseks peate looma Azure AI Agent Service'i. Soovitatav on kasutada [seda malli](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json), et see otse luua (***Märkus:*** Azure AI Agent Service on praegu saadaval piiratud piirkondades. Soovitatav on viidata [sellele lingile](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support), et määrata piirkond).

Agent vajab juurdepääsu SERPAPI-le. Soovitatav on registreeruda [selle lingi](https://serpapi.com/searches) kaudu. Pärast registreerimist saate unikaalse API KEY ja ENDPOINT-i.


# Seadistamine

Selle märkmiku käivitamiseks veendu, et vajalikud teegid on paigaldatud, käivitades käsu `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Teie Semantic Kernel'i versioon peaks olema vähemalt 1.27.2.


Laadige oma .env-faili seaded ja ressursid, veenduge, et olete lisanud oma võtmed ja seaded ning loonud kohaliku .env-faili.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Logi sisse Azure'i

Nüüd pead Azure'i sisse logima. Ava terminal ja käivita järgmine käsk:

```bash
az login
```

See käsk palub sul sisestada oma Azure'i mandaadid, mis võimaldavad Azure AI Agent teenusel korrektselt töötada.


# Selgitus:
See on muutuja, mis salvestab API võtme, et pääseda ligi SERP (otsingumootori tulemuste lehe) API teenusele. API võti on unikaalne identifikaator, mida kasutatakse teie kontoga seotud päringute autentimiseks.

Eesmärk: Selle rea eesmärk on salvestada API võti muutujasse, et seda saaks kasutada päringute autentimiseks SERP API teenuses. API võti on vajalik teenusele ligipääsuks ja otsingute tegemiseks.
Kuidas saada SERP API võtit: SERP API võtme saamiseks järgige neid üldisi samme aadressil https://serpapi.com (täpsed sammud võivad varieeruda sõltuvalt kasutatavast SERP API teenusest):

Valige SERP API teenus: Saadaval on mitmeid SERP API teenuseid, nagu SerpAPI, Google Custom Search JSON API ja teised. Valige see, mis kõige paremini vastab teie vajadustele.

Registreeruge kontole: Minge valitud SERP API teenuse veebilehele ja registreeruge kontole. Teil võib olla vaja esitada mõningaid põhiandmeid ja kinnitada oma e-posti aadress.

Looge API võti: Pärast registreerumist logige oma kontole sisse ja navigeerige API sektsiooni või juhtpaneelile. Otsige võimalust uue API võtme loomiseks või genereerimiseks.
Kopeerige API võti oma .env faili.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Selgitus:
BASE_URL: See on muutuja, mis salvestab SERP API lõpp-punkti põhiaadressi. Muutuja nimi BASE_URL on tavapärane viis näidata, et see URL on lähtepunkt API päringute tegemiseks.  
'https://serpapi.com/search':  

See on tegelik URL-i string, mis on määratud BASE_URL muutujale. See esindab lõpp-punkti, mille kaudu saab teha otsingupäringuid SERP API abil.  

# Eesmärk:
Selle rea eesmärk on määratleda konstant, mis sisaldab SERP API põhiaadressi. Seda URL-i kasutatakse lähtepunktina API päringute koostamiseks, et teostada otsingutoiminguid.  

# Kasutamine:
Määratledes põhiaadressi muutujas, saab seda koodi sees korduvalt kasutada, kui on vaja teha päringuid SERP API-le. See muudab koodi paremini hallatavaks ja vähendab vigade riski, mis võivad tekkida URL-i mitmekordsest käsitsi sisestamisest. Praegune näide on https://serpapi.com/search?engine=bing, mis kasutab Bing otsingumootori API-d. Erinevaid API-sid saab valida aadressil https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Selgitus:

Siin asub teie plugina kood.

Klassi definitsioon: `class BookingPlugin`: Määratleb klassi nimega BookingPlugin, mis sisaldab meetodeid hotellide ja lendude broneerimiseks.

Hotelli broneerimise meetod:

- `@kernel_function(description="booking hotel")`: Dekoraator, mis kirjeldab funktsiooni kui tuumfunktsiooni hotellide broneerimiseks.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Määratleb meetodi hotellide broneerimiseks koos annotatsioonidega parameetrite ja tagastustüübi jaoks.

Meetod koostab parameetrite sõnastiku hotelli broneerimise päringu jaoks ja saadab GET-päringu SERP API-le. See kontrollib vastuse staatust ning tagastab hotelli omadused, kui päring õnnestus, või None, kui päring ebaõnnestus.

Lennu broneerimise meetod:

- `@kernel_function(description="booking flight")`: Dekoraator, mis kirjeldab funktsiooni kui tuumfunktsiooni lendude broneerimiseks.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Määratleb meetodi lendude broneerimiseks koos annotatsioonidega parameetrite ja tagastustüübi jaoks.

Meetod koostab parameetrite sõnastikud väljuva ja tagasilennu päringute jaoks ning saadab GET-päringud SERP API-le. See kontrollib vastuse staatust ja lisab lennuinfo tulemuse stringi, kui päring õnnestus, või kuvab veateate, kui päring ebaõnnestus. Meetod tagastab tulemuse stringi, mis sisaldab lennuinfot.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Selgitus:
Import-lauseid: Impordi vajalikud moodulid Azure'i mandaadi, AI-agendi, vestlussõnumi sisu, autori rolli ja kernel-funktsiooni dekoraatori jaoks.

Asünkroonne kontekstihaldur: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): See loob asünkroonse kontekstihalduri, et hallata Azure'i mandaate ja luua AI-agendi klient.

Agendi nimi ja juhised:
- `AGENT_NAME = "BookingAgent"`: Määratleb agendi nime.
- `AGENT_INSTRUCTIONS = """..."""`: Annab agentidele üksikasjalikud juhised, kuidas broneerimistaotlusi käsitleda.

Agendi määratluse loomine: `agent_definition = await client.agents.create_agent(...)`: Loob agendi määratluse koos määratud mudeli, nime ja juhistega.

AzureAI agendi loomine: `agent = AzureAIAgent(...)`: Loob AzureAI agendi, kasutades klienti, agendi määratlust ja määratletud pistikprogrammi.

Teema loomine: `thread: AzureAIAgentThread | None = None`: Loob agendi jaoks teema. Teema loomine ei ole kohustuslik - kui väärtuseks on määratud `None`, luuakse esimese kutse ajal uus teema ja tagastatakse vastuse osana.

Kasutaja sisendid: `user_inputs = ["..."]`: Määratleb kasutaja sisendite loendi, mida agent peab töötlema.

Lõpuks plokis kustutatakse teema ja agent, et ressursid vabastada.


# Autentimine

`DefaultAzureCredential` klass on osa Azure SDK-st Pythonile. See pakub vaikimisi viisi Azure'i teenustega autentimiseks. Klass proovib autentida mitmel meetodil kindlas järjekorras, näiteks keskkonnamuutujate, hallatud identiteedi ja Azure CLI mandaadi abil.

Asünkroonsed operatsioonid: aio moodul näitab, et `DefaultAzureCredential` klass toetab asünkroonseid operatsioone. See tähendab, et saate seda kasutada koos asyncio-ga, et teostada mitteblokeerivaid autentimispäringuid.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
